In [3]:
def fn_logistic(df_train,df_test):

    # Importing libraries
    import numpy as np
    import os
    import pickle
    import pandas as pd
    import h2o
    from h2o.estimators import H2OGeneralizedLinearEstimator
    from h2o.grid.grid_search import H2OGridSearch
    ! pip install category_encoders # This is to use Target Encoders. This will update Sk-learn as well
    from category_encoders import TargetEncoder








    """
    ------------------------------------------------

    DEFINING VARIABLES

    ------------------------------------------------

    """

    vars_ind_numeric = ['a04', 'a05','a06','a07','a08','a09','a11','a14','a15','b01','b05','b06','c01','c03','d01','d02','d03',
    'e02','e04','e05','e06','e07','e08','e09','e12','e15','e16','e23','f01','f02','f06','f08','f11','f13','f15','f16','f17','f18',
    'f19','f20','f21','f22','f23','f24','f25','f26','f28','f31','f32','unique_id']

    vars_ind_categorical = ['a01','a02','a03','a10','a12','a13','a16','a17','a18','a19','a20','b02','b03','b04','b07','c02','c04',
     'c05','c06','c07','c08','c09','e01','e03','e11','e13','e14','e21','e22','e24','e25','e17','e18','e19','e20','f03','f04','f05',
     'f07', 'f09','f27','f29','f30','f33','f34','f10']

    vars_notToUse = ['unique_id']

    var_dep = ['target']

    vars_ind_hccv = ['e17', 'e18', 'e19', 'f10']


    """
    ------------------------------------------------

    ######################################################################################
    SPLINES:
    ######################################################################################

    Splines have been created for three variables ['f02', 'f11', 'f13'].
    I ran a Random Forest algorithm considering only numeric variables to see which all numeric variables are the most important.
    I did not make splines on all numeric variables, as making splines for all variables could not improve the 
    performance significantly but made the model quite complex to comprehend. Therefore, it was a trade-off between 
    accuracy and complexity.

    For running the random forest, I made a "Sample" dataframe and removed -99 (NAs). I have treated -99 on train and test dataframe
    afterwards through H20. The reason why I did not treat NAs right now was because few of NAs are being taken as NAs in H20 while few of them were 
    been taken as "NA" (String) which was strange. Therefore I treated all NAs in H20 only.

    Also, I did not standarised the variables, because Random Forest (for splines) do not require standardising variables 
    while H20 will automatically treat them while doing Logistic Regression.

    I have run the spline function on test as "fn_tosplines(df_test[var])" because what I feels is, in real life scenario, we
    wouldn't be having the test frame and therefore have to use the pre-decided variables and percentile values on test frame.



    ######## The Following is the code that gave us most important numeric feature through Random Forest ########

    from sklearn.datasets import make_classification
    from sklearn.ensemble import RandomForestClassifier
    from matplotlib import pyplot

    sample = df_train.copy() 
    sample = sample.replace(-99, np.nan)
    sample = sample.dropna()

    # random forest for feature importance on a classification problem
    # define dataset
    X, y = sample[[var for var in vars_ind_numeric if var not in vars_notToUse]], np.array(sample[var_dep])
    # define the model
    model = RandomForestClassifier()
    # fit the model
    model.fit(X, y)
    # get importance
    importance = model.feature_importances_
    # summarize feature importance
    for i,v in enumerate(importance):
        print('Feature: %0d, Score: %.5f' % (i,v))
    # plot feature importance
    pyplot.bar([x for x in range(len(importance))], importance)
    pyplot.show()

    high_imprtance_num_var = list(X.columns)
    high_imprtance_num_var  = [high_imprtance_num_var [i] for i in [29,32,33]]

    ################## END ##################

    ------------------------------------------------

    """
    high_imprtance_num_var = ['f02', 'f11', 'f13']


    vars_ind_tospline = high_imprtance_num_var

    def fn_tosplines(x):
        x = x.values
        # hack: remove zeros to avoid issues where lots of values are zero
        x_nonzero = x[x != 0]
        ptiles = np.percentile(x_nonzero, [10, 20, 40, 60, 80, 90])
        #print(var, ptiles)
        df_ptiles = pd.DataFrame({var: x})
        for idx, ptile in enumerate(ptiles):
            df_ptiles[var + '_' + str(idx)] = np.maximum(0, x - ptiles[idx])
        return(df_ptiles)

    ## Splines for Train set
    for var in vars_ind_tospline:
        df_ptiles = fn_tosplines(df_train[var])
        df_train.drop(columns=[var], inplace=True)
        vars_ind_numeric.remove(var)
        df_train = pd.concat([df_train, df_ptiles], axis=1, sort=False)
        vars_ind_numeric.extend(df_ptiles.columns.tolist())


    ## Splines for Test set
    for var in vars_ind_tospline:
        df_ptiles_t = fn_tosplines(df_test[var])  # Splines on train set has been used
        df_test.drop(columns=[var], inplace=True)
        df_test = pd.concat([df_test, df_ptiles_t], axis=1, sort=False)




    """
    ------------------------------------------------
    ######################################################################################
    TREATING NAs : Justification for using Mean imputes instead of Skip method:
    ######################################################################################
    
    H20 documentation for GLM suggest that it automatically mean imputes the NAs while the another method is removing 
    the observations that have NAs. The document also suggest that if, we have few columns with many NAs, we might accidentally 
    be losing all our rows so its better to exclude (skip) them. While on the other hand, if we have many columns with a small
    fraction of uniformly distributed missing values where every row is likely to have at least one missing value. In this case, 
    impute the NAs is suggested (e.g., substitute the NAs with mean values) before modeling.

    Through our code (attached below) we can see in the test frame there are many rows with few NAs and thus if we use Skip
    method as we might loose information. 
    Therefore we have used Mean imputation method (Although Skip method showed a better accuracy on Train frame).

    NAs will be treated in the H20 Frame, right now we are only removing 'C02' variable as it has got >50% of NAs.



    ################## Code ##################

    # To search if there are any categorical variables with NAs where NA == -99
    (df_train[vars_ind_numeric] == -99).astype(int).sum(axis=0)
    # Yes there are a lot of variables with NAs, in some of them NAs are near to 100,000 values

    # To search if there are any categorical variables with NAs
    df_test[vars_ind_categorical].isna().sum(axis = 0)
    # Yes, there are a lot of categorical varibales with NAs

    ################## END ##################

    ------------------------------------------------
    """


    # Since C02 has a lot of NAs (>50%), we will drop it from both train and test
    df_test.drop('c02', axis=1, inplace = True)
    df_train.drop('c02', axis=1, inplace = True)
    vars_ind_categorical.remove('c02')





    """
    ------------------------------------------------
    ######################################################################################
    CARDINALITY: 
    ######################################################################################
    
    For treating the hccv we have used Target Encoders (Sk-Learn). We have also observed that there is oversampling of few factors 
    while others are under-sampled therefore we have used smoothing factor = 4. Smoothing of 4 is chosen by following few online 
    blogs. Also scikit learn is used to do the target encoding because I was a bit confused with H20 target encoding. Additionally,
    H20 automatically perform one_hot encoding on categorical variables so they have not been treated.
    Note: I needed to update the scikit-learn and category_ecoders library

    ------------------------------------------------
    """

    # Target encoders on Train
    enc = TargetEncoder(cols=vars_ind_hccv, smoothing =4)
    enc.fit_transform(df_train, df_train['target'])
    df_train = enc.transform(df_train, df_train['target'])


    # Target encoders on Test
    df_test['target'] = np.nan # Creating dummy 'target' variable for using the the enc.transform function
    df_test = enc.transform(df_test) # applying the already trained encoder
    df_test.drop(columns=['target'], inplace=True) # Dropping dummy 'target' variable



    # Initiating h20
    # h2o.init(port=54321)
    h2o.init(port=54321, max_mem_size = "14g") # Asking h20 to use 14 GB of Ram
    h2o.connect()


    """
    ------------------------------------------------
    ######################################################################################
    H20 FRAMES:
    ######################################################################################
    We have removed 'unique_id' from Frames as it was not useful in prediction

    ------------------------------------------------
    """

    # H20 Frames: 
    vars_to_use = vars_ind_numeric + vars_ind_categorical
    vars_to_use.remove('unique_id')
    vars_ind_numeric.remove('unique_id')


    h2o_df_train = h2o.H2OFrame(df_train[[var for var in vars_to_use+var_dep ]], destination_frame = 'df_train') # Train Frame
    h2o_df_test  = h2o.H2OFrame(df_test[[var for var in vars_to_use]], destination_frame = 'df_test')  # Test Frame



    """
    ------------------------------------------------
    ######################################################################################
    TREATING NAs:
    ######################################################################################
    
    Converting -99 to NAs. We have not done this before becuase as discussed earlier, converting pandas NAs to H20 Frame
    were not consistent and was giving an error (may be due to the older version of H20 in the image).

    ------------------------------------------------
    """

    # Converting -99 to NA in train
    for var in vars_ind_numeric:
        h2o_df_train[h2o_df_train[var] == -99.0 , var] = None


    # Converting -99 to NA in test
    for var in vars_ind_numeric:
        h2o_df_test[h2o_df_test[var] == -99.0 , var] = None



    # H20 Document suggest to make dependent variable as factor for classification task
    h2o_df_train[var_dep] = h2o_df_train[var_dep].asfactor()




    """
    ------------------------------------------------
    ######################################################################################
    INTERACTIONS:
    ######################################################################################
    
    Calculating glm with interactions increases the kaggle score from 77% to 83%.

    I have not used the interaction on all variables, instead I have used it only on 2 variables ('f03' and 'e11'). These two 
    variables have been chosen after running GLM model and then calculating variables importance.
    I did not use all categorical variables, as it could not improve the performance significantly but made the
    model quite complex to comprehend. Therefore, it was a trade-off between accuracy and complexity.




    ################## CODE ##################

    # Code for running GLM to see which all variables are important

    Idea has been taken from: https://aichamp.wordpress.com/2017/09/29/python-example-of-building-glm-gbm-and-random-forest-
    binomial-model-with-h2o/


    from h2o.estimators.glm import H2OGeneralizedLinearEstimator
    glm_logistic = H2OGeneralizedLinearEstimator(family = "binomial")
    glm_logistic.train(x=vars_to_use , y= 'target', training_frame=h2o_df_train, model_id="glm_logistic")
    preds = glm_logistic.predict(h2o_df_test)
    df_test['Predicted'] = np.round(preds[2].as_data_frame(), 5)
    df_preds_dt = df_test[['unique_id', 'Predicted']].copy()
    df_test[['unique_id', 'Predicted']].to_csv(dirPOutput + '1st.csv', index=False)
    log_var_imp = glm_logistic.varimp(use_pandas=True).head()
    log_var_imp.loc[0:5, 'variable'].tolist()

    Output : ['f10', 'f03.F', 'e19', 'e11.A', 'f03.E']

    ################## END ##################


    ------------------------------------------------
    """



    # I have chosen min_occurence as int(len(h2o_df_train)/40) after many trial and error.
    # With int(len(h2o_df_train)/40) on 250K train data I was getting 5 factors for each variable. I believe performing 
    # interactions on top 4-5 variables rather than all the variables having occurrence > 10/20 would make the model
    # faster and more interpretable
    
    # Train Frame
    interaction_frame_train = h2o_df_train.interaction(['f03', 'e11'], pairwise = False, max_factors = 100,
                                                       min_occurrence = int(len(h2o_df_train)/40))
    # Test Frame
    interaction_frame_test = h2o_df_test.interaction(['f03', 'e11'], pairwise = False, max_factors = 100, 
                                                     min_occurrence = int(len(h2o_df_train)/40))


    # Cbinding interaction frame to train and test
    h2o_df_train = h2o_df_train.cbind(interaction_frame_train)
    h2o_df_test = h2o_df_test.cbind(interaction_frame_test)

    # incuding interaction frame's variable to variables list
    vars_to_use = vars_to_use + ['f03_e11']
    vars_ind_numeric = vars_ind_numeric + ['f03_e11']





    """
    ------------------------------------------------


    ######################################################################################
    Running Grid Search for hyper parameters
    ######################################################################################
    The thought process of selecting each parameters and its values has been discussed below
    
    # * We could have added 'missing_values_handling': ["skip", "mean_imputation"] to the  hyper-parameters, but as discussed
    above, predicting with 'Skip' method may give us a better accuracy on train, but will give us a bad result on Test
    # * We have not included the 'Standarised' parameter because it is True by default and wants to keep it that way.
    # * Search Strategy: The default strategy, "Cartesian", covers the entire space of h-p combinations but takes too much time.
    Therefore we have used "Random Discrete Strategy".
    # * "max_models": 30: taken the idea of 30 models from the lecture notes
    # * stopping_metric": "AUTO": Auto is logloss for classification. For task 2 & 3, stopping_metric is set to AUC
    # * seed = 2020: Seed so that it generates similar model every-time
    # * nfolds = 5: nfolds ideally took more time but I believe it is more efficient in terms of training the algorithm
    rather than taking validation set.
    

    
    

    # Defining hyper parameters to search for

    alpha_opts = np.arange(0, 1, 0.01).tolist()
    hyper_parameters = {"alpha":alpha_opts} 


    # Defining criteria
    criteria = {"strategy": "RandomDiscrete", 
                "max_runtime_secs": 5400,
                "max_models": 30, 
                "stopping_metric": "AUTO", 
                "seed": 2020}

    # Model
    grid = H2OGridSearch(H2OGeneralizedLinearEstimator(family="binomial", # Logit link function
                                                       nfolds = 5, 
                                                       lambda_search=True), 
                         hyper_params=hyper_parameters,
                         grid_id='g1',
                         search_criteria=criteria)


    # Training the model
    grid.train(y = "target",
               x = vars_to_use, 
               training_frame = h2o_df_train
               )


    # Looking the grid results
    grid = grid.get_grid(sort_by='accuracy', decreasing=True)
    bst = grid.models[1] # check if you want 0 or 1
    grid


    # Observing which all variants are the most important for my own understanding. These co-efficient are actually different
    # from what we got from our previous GLM and Random Forest method except f03 is common
    # Normalised coefficients
    bst_coef = bst.coef_norm()
    # {k: v for k, v in sorted(bst_coef.items(), key=lambda item: item[1])}

    # Output: f10, e19, f03, f27, f29

    ------------------------------------------------
    """




    """
    ------------------------------------------------

    ######################################################################################
    Training and fitting the H2OGeneralizedLinearEstimator (GLM)
    ######################################################################################

    The grid search returned 3 models in 90 minutes. We could have waited for the entire process of getting 30 models
    but I didn't as it would have taken 15 hours to do so but I wonder we might get a better result.

    The model returned by the Grid search is as follow and the best model is having alpha = 0.92

          alpha   model_ids             accuracy
    0    [0.92]  g1_model_1             0.745579
    1    [0.49]  g1_model_2   0.7443500000000001
    2    [0.01]  g1_model_3  0.49104800000000004


    ------------------------------------------------
    """


    GLM_model = H2OGeneralizedLinearEstimator(family="binomial", # Logit link function
                                  nfolds = 5, # nfolds ideally took more time but I believe it is the correct 
                                  # way of doing rather than taking validation set.
                                  lambda_search=True, # Lambda is a regularisation object and lambda_search 
                                  alpha = 0.92, # As returned by the grid search
                                  seed = 2020 ,
                                  max_runtime_secs = 5400, # allowed to run for 90 mins
                                  )


    GLM_model.train(y = "target",
               x = vars_to_use, 
               training_frame = h2o_df_train
               )
    
    
    # Predicting the test values
    preds = GLM_model.predict(h2o_df_test) # previously written grid
    df_test['Predicted'] = np.round(preds[2].as_data_frame(), 5)
    df_preds_dt = df_test[['unique_id', 'Predicted']].copy()
    result_df_test = df_test[['unique_id', 'Predicted']]


    return(GLM_model, result_df_test)

In [ ]:
fn_logistic(df_train,df_test)